In [1]:
import numpy as np
import scipy.io as sio 
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.cuda.device_count()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [30]:
N = 5  # number of  nodes

learning_rate = 0.001  # learning rate

batch_size = 1000
n_batch = 100
n_epoch = 1000  # number of iterations
n_val = 10000   # number of validation data
n_test = 10000

mode = 'mlp'

if mode == 'mlp':
    n_hidden = 1000
    n_layer = 10
    T = 1
    model_location = "./UL_MLP_N%d_(nH%d,nL%d)" % (N, n_hidden, n_layer)
elif mode == 'cnn':
    n_kernel = 200
    T = 5
    model_location = "./UL_CNN_N%d_(nK%d, T%d)" % (N, n_kernel, T)
    
if not os.path.isdir(model_location): os.mkdir(model_location)
    
fw = 1-torch.eye(N).unsqueeze(2).tile(1, 1, N).to(device)
gw = 1-torch.eye(N).unsqueeze(1).tile(1, N, 1).to(device)
window = {'a': fw, 'r': gw}

In [31]:
class MLP(nn.Module):
    def __init__(self, n_hidden, n_layers):
        super(MLP, self).__init__()
        
        layers = [nn.BatchNorm1d(N**2), nn.Linear(N**2, n_hidden), nn.BatchNorm1d(n_hidden), nn.SELU()]
        
        for i in range(n_layer-2):
            layers.append(nn.Linear(n_hidden, n_hidden))
            layers.append(nn.BatchNorm1d(n_hidden))
            layers.append(nn.SELU())
            
        layers.append(nn.Linear(n_hidden, 2*N**2))
        #layers.append(nn.BatchNorm1d(2*N**2))
        #layers.append(nn.Tanh())
                                      
        self.layers = nn.Sequential(*layers)
                                       
    def forward(self, x, C):
        x = self.layers(x)
        #x = torch.max(torch.abs(C).view(-1, N**2), 1).values.unsqueeze(1)*x
        return x


In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv = nn.ModuleList([self.layer(t) for t in range(2*T)])

    def layer(self, t):
        if t%2 == 0:
            conv0 = nn.Conv2d(2, 1, (N, 1))
        else:
            conv0 = nn.Conv2d(2, 1, (1, N))
        conv1 = nn.Sequential(nn.Conv2d(3, n_kernel, 1), nn.BatchNorm2d(n_kernel), nn.ReLU(),
                              nn.Conv2d(n_kernel, n_kernel, 1), nn.ReLU(),
                              nn.Conv2d(n_kernel, 1, 1), nn.ReLU())
        conv2 = nn.Sequential(nn.Conv2d(3, n_kernel, 1), nn.BatchNorm2d(n_kernel), nn.ReLU(),
                              nn.Conv2d(n_kernel, n_kernel, 1), nn.ReLU(),
                              nn.Conv2d(n_kernel, 1, 1))
        return nn.ModuleList([conv0, conv1, conv2])
        
    def module(self, x, y, C, layer, name_scope):
        if name_scope == 'r':
            x_init = layer[0](torch.cat([C, y], 1)).tile(1, 1, N, 1)
        elif name_scope == 'a':
            x_init = layer[0](torch.cat([C, y], 1)).tile(1, 1, 1, N)
        x = layer[1](torch.cat([C, x, x_init], 1)).reshape(-1, 1, N, N)
        x = torch.where(window[name_scope] == 0.0, torch.tensor([-10000.0]).to(device), x)
        if name_scope == 'r':
            x = layer[2](torch.cat([C, torch.max(x, 2).values.unsqueeze(1), x_init], 1))
        elif name_scope == 'a':
            x = layer[2](torch.cat([C, torch.max(x, 3).values.permute(0, 2, 1).unsqueeze(1), x_init], 1))
        #x = torch.max(torch.abs(C).reshape(-1, N**2), 1).values.unsqueeze(1).unsqueeze(2).unsqueeze(3)*x
        #Cmax = torch.max(torch.abs(C).reshape(-1, N**2), 1).values.reshape(-1, 1, 1, 1)
        #x = torch.maximum(torch.minimum(x, Cmax), -Cmax)
        return x
                                               
    def forward(self, C):
        
        x = torch.zeros(C.shape[0], N, N, 2, T).to(device)
        a, r = C, C
        
        for t in range(0, 2*T, 2):
            
            r = self.module(a, r, C, self.conv[t], 'r')
            a = self.module(r, a, C, self.conv[t+1], 'a')
            
            x[:, :, :, :, int(t/2)] = torch.stack([r.squeeze(), a.squeeze()], 3)

        return x
    


In [32]:
if mode == 'mlp':
    model = MLP(n_hidden, n_layer).to(device)
elif mode == 'cnn':
    model = CNN().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()

In [6]:
def computeStatistics(y, C):
    ff = (y[:, :, :, 1] + C/2).unsqueeze(1).tile(1, N, 1, 1)
    gg = (y[:, :, :, 0] + C/2).unsqueeze(1).tile(1, N, 1, 1)

    ff = torch.where(fw == 0.0, torch.tensor([-10000.0]).to(device), ff)
    gg = torch.where(gw == 0.0, torch.tensor([-10000.0]).to(device), gg)
       
    f = C/2 - torch.max(ff, 2).values 
    g = C/2 - torch.max(gg, 3).values.permute(0, 2, 1)

    return f, g

def computeCost(y, C):
    x = torch.max(y[:, :, :, 0] + y[:, :, :, 1], 1).indices
    x_one_hot = F.one_hot(x, num_classes=N)
    cost = (C*x_one_hot.permute(0, 2, 1)).sum()/y.shape[0]
    
    return x, x_one_hot, cost

In [ ]:
C_val = torch.from_numpy(np.random.exponential(scale=1, size=(n_val, N, N))).float().to(device)
c_val = 1000000.0
cost_eval = np.zeros((n_epoch, 4))

for epoch in range(n_epoch):
    
    model.train()
    
    for batch in range(n_batch):
        
        #C_train = torch.rand(batch_size, N, N).to(device)
        C_train = torch.from_numpy(np.random.exponential(scale=1, size=(n_batch, N, N))).float().to(device)
        
        if mode == 'mlp':
            y = model(torch.reshape(C_train, (-1, N**2)), C_train).reshape(-1, N, N, 2, 1)
        elif mode == 'cnn':
            y = model(C_train.unsqueeze(1))
            
        loss = 0.0
        for t in range(T):
            yt = y[:, :, :, :, t]
            f, g = computeStatistics(yt, C_train)
            loss += loss_fn(yt, torch.stack([f, g], dim=3))/T
        
        _, _, cost_train = computeCost(yt, C_train)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost_eval[epoch, 0] += loss.item()/n_batch
        cost_eval[epoch, 2] += cost_train/n_batch
        
    model.eval()
    
    with torch.no_grad():
        if mode == 'mlp':
            y_val = model(torch.reshape(C_val, (-1, N**2)), C_val).reshape(-1, N, N, 2, 1)
        elif mode == 'cnn':
            y_val = model(C_val.unsqueeze(1))
            
        for t in range(T):
            yt_val = y_val[:, :, :, :, t]
            f_val, g_val = computeStatistics(yt_val, C_val)
            cost_eval[epoch, 1] += loss_fn(yt_val, torch.stack([f_val, g_val], dim=3)).item()/T
        #f_val, g_val = computeStatistics(y_val, C_val)
        #cost_eval[epoch, 1] = loss_fn(y_val, torch.stack([f_val, g_val], dim=3)).item()
        
        x_val, _, cost_val = computeCost(yt_val, C_val)
        cost_eval[epoch, 3] += cost_val
        
    if (epoch + 1) % 1 == 0:
        print('epoch:%d ' % (epoch + 1),\
              'train MSE:%0.5f, ' % cost_eval[epoch, 0],\
              'validation MSE:%0.5f, ' % cost_eval[epoch, 1],\
              'train COST: %0.5f, ' % cost_eval[epoch, 2],\
              'validation COST:%0.5f, ' % cost_eval[epoch, 3])
        
    if cost_eval[epoch, 1] < c_val:
        print('Best validation MSE:%0.5f at epoch %d' % (cost_eval[epoch, 1], epoch + 1),\
              '(train MSE:%0.5f)' % cost_eval[epoch, 0])
        c_val = cost_eval[epoch, 1]
        save_dict = {'model': model.state_dict()}
        torch.save(save_dict, model_location + "/model.pth")

sio.savemat(model_location+"/result.mat", \
            {'trainMSE': cost_eval[:, 0], 'validationMSE': cost_eval[:, 1], \
             'trainCOST': cost_eval[:, 2], 'validationCOST': cost_eval[:, 3]})
        
    

In [8]:
np.random.seed(170835)
C_test = torch.from_numpy(np.random.exponential(scale=1, size=(n_test, N, N))).float().to(device)

checkpoint = torch.load(model_location+'/model.pth')

if mode == 'mlp':
    model = MLP(n_hidden, n_layer).to(device)
elif mode == 'cnn':
    model = CNN().to(device)
    
model.load_state_dict(checkpoint['model'])
model.eval()

with torch.no_grad():
    if mode == 'mlp':
        y_test = model(torch.reshape(C_test, (-1, N**2)), C_test).reshape(-1, N, N, 2, 1)
    elif mode == 'cnn':
        y_test = model(C_test.unsqueeze(1))
        
    f_test, g_test = computeStatistics(y_test[:, :, :, :, -1], C_test)
    loss_test = loss_fn(y_test[:, :, :, :, -1], torch.stack([f_test, g_test], dim=3)).item()
        
    x_test, x_test_one_hot, cost_test = computeCost(y_test[:, :, :, :, -1], C_test)

print(f'Test MSE: {loss_test}, Test Cost: {cost_test}')

Test MSE: 0.056223221123218536, Test Cost: 10.175226211547852


In [9]:
infeasible_ratio = 0

for i in range(n_test):
    if torch.unique(x_test[i, :]).shape[0] < N:
        infeasible_ratio += 1/n_test
    
print(infeasible_ratio)

0.5103999999999601


In [10]:
yy = y_test[0, :, :, 0, -1] + y_test[0, :, :, 1, -1]
print(torch.max(yy, 1).indices)
print(torch.max(yy, 0).indices)
print(yy)

tensor([3, 0, 2, 4, 0], device='cuda:0')
tensor([4, 4, 2, 0, 3], device='cuda:0')
tensor([[-4.4675e+00, -2.5561e+00, -2.4517e+00,  9.6333e-01, -1.7470e+00],
        [ 5.7526e-02, -1.2051e+00, -1.8346e+00, -2.6447e+00,  4.6558e-02],
        [-2.8331e+00, -1.9017e+00,  1.0758e-01, -2.7167e+00, -5.6163e-01],
        [-2.9648e+00,  9.9970e-04, -1.6046e+00, -1.1874e+00,  7.3278e-02],
        [ 2.8570e-01,  1.0269e-01, -2.0164e+00, -2.4621e+00, -1.3240e+00]],
       device='cuda:0')


In [11]:
print(x_test[0, :])

tensor([4, 4, 2, 0, 3], device='cuda:0')


In [12]:
ff = (y_test[:, :, :, 1] + C_test/2).unsqueeze(1).tile(1, N, 1, 1)
gg = (y_test[:, :, :, 0] + C_test/2).unsqueeze(1).tile(1, N, 1, 1)

ff = torch.where(fw == 0.0, torch.tensor([-10000.0]).to(device), ff)
gg = torch.where(gw == 0.0, torch.tensor([-10000.0]).to(device), gg)
       
f = C_test/2 - torch.max(ff, 2).values 
g = C_test/2 - torch.max(gg, 3).values


RuntimeError: The size of tensor a (5) must match the size of tensor b (10000) at non-singleton dimension 1

In [ ]:
print(gg)

In [ ]:
torch.max(C_train.view(-1, N**2), 1).values.unsqueeze(1).shape

In [ ]:
y.shape